In [ ]:
#default_exp cactus
#export
from torchmeta.datasets.helpers import miniimagenet, omniglot
from torchmeta.utils.data import BatchMetaDataLoader
from torchmeta.utils.gradient_based import gradient_update_parameters
from torchmeta.modules import MetaModule, MetaSequential, MetaConv2d, MetaBatchNorm2d, MetaLinear

import kornia as K
import higher

import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchnet
import tqdm

from collections import OrderedDict
from copy import deepcopy

from unsupervised_meta_learning.nn_utils import Flatten

In [ ]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [ ]:
#export

def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )

In [ ]:
#export

class ProtoNet(nn.Module):
    def __init__(self, encoder=None, **kwargs):
        super().__init__()
        self.encoder = encoder
        if encoder is None:
            x_dim = kwargs['x_dim']
            hid_dim = kwargs['hid_dim']
            self.encoder = nn.Sequential(
                conv_block(x_dim, hid_dim),
                conv_block(hid_dim, hid_dim),
                conv_block(hid_dim, hid_dim),
                conv_block(hid_dim, hid_dim),
                Flatten()
            )
        
    def loss(self, sample):
        xs = sample['train']
        xq = sample['test']

        # might have to change this bit to catch the correct shapes and sizes
        n_class = xs.shape[0]
        n_support = xs.shape[1]
        n_query = xq.shape[1]
        
        target_inds = torch.arange(0, n_class).view(n_class, 1, 1).expand(n_class, n_query, 1).long()
        
        if xq.is_cuda:
            target_inds = target_inds.to('cuda')
            
        x = torch.cat([
            xs.view(n_class * n_support, *xs.size()[2:] ),
            xq.view(n_class * n_query, *xq.size()[2:])
        ], 0)
        
        z = self.encoder(x)
        z_dim = z.size(-1)
        
        z_proto = z[:n_class * n_support].view(n_class, n_support, z_dim).long()
        zq = z[n_class * n_support:]
        
        dists = torch.cdist(zq, z_proto)
        
        log_p_y: torch.tensor = F.log_softmax(-dists, dim=1).view(n_class, n_query, -1)
        
        loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
        
        _, y_hat = log_p_y.max(2)
        
        acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()
        
        return loss_val, {
            'loss': loss_val.item()
            'acc': acc_val.item()
        }
        
        

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 01_nn_utils.ipynb.
Converted 02_maml.ipynb.
Converted 03_cactus.ipynb.
Converted index.ipynb.
